<a href="https://colab.research.google.com/github/tantowijh/colab-collection/blob/main/TorrentDownloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####**Torrent Downloader**



###**Start Installing Torrent**

---

Run this command to before start downloading

In [ ]:
#@markdown Install torrent
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent
!pip install termcolor
 
import libtorrent as lt
from queue import Queue
from threading import Thread
import time
import os
import datetime
from IPython.display import display
from IPython.display import clear_output
import ipywidgets as widgets
from tqdm.notebook import tqdm
from IPython.display import clear_output

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

clear_output()

In [ ]:

#@markdown *This cell runs a JS code that automatically reconnects you to avoid inactivity due to idle.*

import IPython
from IPython.display import clear_output
from google.colab import output

display(IPython.display.Javascript('''
 function connectRefresher() {
       window.ConnectButtonIntervalId = setInterval(function ConnectButton(){
                console.log("connected"); 
                document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
                document.querySelector("colab-sessions-dialog").shadowRoot.querySelector("#footer > div > paper-button").click();
                console.log("closed the dialog!!"); 
            },60000);
    }
    
 function clearRefresher() { 
           console.log("clear Interval called !!");
           clearInterval(window.ConnectButtonIntervalId);
    }

 connectRefresher(); //to connect the refresher
 clearRefresher(); //to disconnect the refresher
'''))

clear_output()

###**Mount Google Drive**

---
Run this command to coonect a google drive, so you can save the file directly to google drive.


In [ ]:
#@markdown Connect google drive
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


###**Torrent Downloader**

---
Choose your favorite download method, by a torrent file or magic-links.


In [ ]:
#@markdown <h4><b>Add from torrent file</b></h4>
DownloadLocation = "/content" #@param {type:"string"}

from google.colab import files

from pathlib import Path
Path(DownloadLocation).mkdir(parents=True, exist_ok=True)

source = files.upload()
params = {
    "save_path": str(DownloadLocation),
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

In [ ]:
#@markdown <h4><b>Add from magic link</b></h4>
DownloadLocation = "/content/drive/Shareddrives/06. YT-DLP" #@param {type:"string"}

from pathlib import Path
Path(DownloadLocation).mkdir(parents=True, exist_ok=True)

params = {"save_path": str(DownloadLocation)}


while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )

Enter Magnet Link Or Type Exit: magnet:?xt=urn:btih:56be86be2480fc7425f63866fa68d2da73a0002d&dn=%5BReinForce%5D%20Strike%20the%20Blood%20final%20batch%20%28BDRip%201920x1080%20x264%20FLAC%29&tr=http%3A%2F%2Fnyaa.tracker.wf%3A7777%2Fannounce&tr=udp%3A%2F%2Fopen.stealth.si%3A80%2Fannounce&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2Fexodus.desync.com%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.torrent.eu.org%3A451%2Fannounce
Enter Magnet Link Or Type Exit: exit


magnet:?xt=urn:btih:56be86be2480fc7425f63866fa68d2da73a0002d&dn=%5BReinForce%5D%20Strike%20the%20Blood%20final%20batch%20%28BDRip%201920x1080%20x264%20FLAC%29&tr=http%3A%2F%2Fnyaa.tracker.wf%3A7777%2Fannounce&tr=udp%3A%2F%2Fopen.stealth.si%3A80%2Fannounce&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2Fexodus.desync.com%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.torrent.eu.org%3A451%2Fannounce

Delete it after complete!

In [ ]:
#@markdown Start download
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)

FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…